### Train VGG LSTM Attention Multiclass Model

Reference: https://learn.microsoft.com/en-us/azure/machine-learning/how-to-train-pytorch?view=azureml-api-2

In [14]:
# Import necessary libraries
from dotenv import load_dotenv
from datetime import datetime
import os

# Load the .env file
load_dotenv()

True

In [15]:
# Azure ML specific imports
from azure.ai.ml import MLClient
from azure.identity import DefaultAzureCredential
from azure.ai.ml.entities import AmlCompute
from azure.ai.ml import command

# Configure the Azure workspace and authentication
subscription_id = os.getenv("AZURE_SUBSCRIPTION_ID")
resource_group_name = os.getenv("AZURE_RESOURCE_GROUP")
workspace_name = os.getenv("AZURE_WORKSPACE_NAME")

credential = DefaultAzureCredential()
ml_client = MLClient(
    credential=credential,
    subscription_id=subscription_id,
    resource_group_name=resource_group_name,
    workspace_name=workspace_name,
)

# Create or get the GPU cluster
gpu_compute_target = "gpucluteruk"
try:
    gpu_cluster = ml_client.compute.get(gpu_compute_target)
    print(f"You already have a cluster named {gpu_compute_target}, we'll reuse it as is.")
except Exception:
    print("Creating a new GPU compute target...")
    gpu_cluster = AmlCompute(
        name=gpu_compute_target,
        type="amlcompute",
        size="Standard_ND96amsr_A100_v4",
        min_instances=1,
        max_instances=4,
        idle_time_before_scale_down=180,
        tier="Dedicated",
    )
    gpu_cluster = ml_client.begin_create_or_update(gpu_cluster).result()
print(f"AMLCompute with name {gpu_cluster.name} is created, the compute size is {gpu_cluster.size}")


You already have a cluster named gpucluteruk, we'll reuse it as is.
AMLCompute with name gpucluteruk is created, the compute size is Standard_ND96amsr_A100_v4


In [16]:
# Azure ML environment and job setup
custom_env_name = "custom-acpt-pytorch-113-cuda117:9"



env_vars = {
    'AZURE_STORAGE_ACCOUNT': os.getenv("AZURE_STORAGE_ACCOUNT"),
    'AZURE_STORAGE_KEY': os.getenv("AZURE_STORAGE_KEY"),
    'BLOB_CONTAINER': os.getenv("BLOB_CONTAINER"),
    'PRETRAINED_VGG_MODEL_URI': os.getenv('PRETRAINED_VGG_MODEL_URI')
}

print(env_vars)

def get_display_name(base_name):
    current_time = datetime.now().strftime("%Y-%m-%d %H:%M:%S")
    display_name = f"{base_name} {current_time}"
    return display_name

experiment_name = "lstm_attention_vgg_multiclass"

inputs = {
        'network': "lstm_attention_vgg_multiclass",
        "num_epochs": 20,
        "batch_size": 16,
        "learning_rate": 0.0005,
        "k": 5,
        "i": 0,
        "dataset": "ccccii"
    }

# Tags
import urllib
parsed_url = urllib.parse.urlparse(os.getenv('PRETRAINED_VGG_MODEL_URI'))
path_parts = parsed_url.path.split('/')
pre_trained_model = path_parts[3].replace('dcid.', '') + '/' + path_parts[5]
tags = {'pretrained_model': pre_trained_model}
tags.update({key: str(value) for key, value in inputs.items()})

job = command(
    inputs=inputs,
    compute=gpu_compute_target,
    environment=custom_env_name,
    code="../",  # location of source code
    command=(
        "python -m scripts.train.train_lstm_attention_vgg_multiclass "
        "--run_cloud "
        "--dataset ${{inputs.dataset}} "
        "--k ${{inputs.k}} "
        "--i ${{inputs.i}} "
        "--num_epochs ${{inputs.num_epochs}} "
        "--batch_size ${{inputs.batch_size}} "
        "--learning_rate ${{inputs.learning_rate}} "
    ),
    environment_variables=env_vars,
    experiment_name=experiment_name,
    display_name=get_display_name(experiment_name),
    tags=tags
)


ml_client.jobs.create_or_update(job)


{'AZURE_STORAGE_ACCOUNT': 'myexperiments0584390316', 'AZURE_STORAGE_KEY': 'GgRchFPTygHq+qqWRn/MLRgvcBL9BXk2Q47maKtDiEMeAyWqhKrdRyxTdFBkdozMyLRJg6WKeK2k+AStH2zfBQ==', 'BLOB_CONTAINER': 'datasets', 'PRETRAINED_VGG_MODEL_URI': 'https://myexperiments0584390316.blob.core.windows.net/azureml/ExperimentRun/dcid.neat_dress_mplb4g5y3g/outputs/vgg_multiclass_19epoch_0.00050lr_0.991rec.pth'}


Experiment,Name,Type,Status,Details Page
lstm_attention_vgg_multiclass,maroon_goat_nf59lw7tt8,command,Starting,Link to Azure Machine Learning studio
